# Beginner's Guide to XGBoost for Classification Problems
## Utilize the hottest ML library for state-of-the-art performance
<img src='images/pexels.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@dom-gould-105501?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Dom Gould</a>
        on 
        <a href='https://www.pexels.com/photo/rear-view-of-silhouette-man-against-sky-during-sunset-325790/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Setup

In [36]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

warnings.filterwarnings("ignore")

### What is XGBoost and why is it so popular?

Let me introduce you to the hottest Machine Learning library in the ML community - XGBoost. In recent years, it has been the main driving force behind the algorithms that win massive ML competitions. Its speed and performance is unparalleled and it consistently outperforms any other algorithms aimed at supervised learning tasks.

The library is parallelizable which means the core algorithm can run on clusters of GPUs or even across a network of computers. This makes it feasible to solve ML tasks by training on hundreds of millions of training examples with high performance. 

Originally, it was written in C++ as a command line application. After winning a huge competition in the field of physics, it started being widely adopted by the ML community. As a result, now the library has its APIs in several other languages including Python, R and Julia. 

In this post, we will learn the fundamentals of XGBoost by solving classification tasks. ## TODO

### Refresher on Terminology

Before we move on to code examples of XGBoost, let's refresh on some of the terms we will be using throughout the post. 

**Classification task**: a supervised machine learning task in which one should predict if an instance is in some category by studying the item's features. For example, by looking at the body measurements, patient history and glucose levels of a person, you can predict whether a person belongs to 'Has diabetes' or 'Does not have diabetes' group.

**Binary classification**: One type of classification where the target instance can only belong to either one of two classes. For example, predicting whether an email is spam or not, whether a customer purchases some product or not, etc.

**Multi-class classification**: Another type of classification problems where the target can belong to one of many categories. For example, predicting the species of a bird, guessing someones bloody type, etc.

If you find yourself confused by other terminology, I have written a small ML dictionary for beginners:

https://towardsdatascience.com/codeless-machine-learning-dictionary-for-dummies-fa912cc7bdfe

### How to prepocess your datasets for XGBoost

Apart from basic data cleaning operations, there are some requirements for XGBoost to achieve top performance. Mainly:
- Numeric features should be scaled
- Categorical features should be encoded

To show how these steps are done, we will be using the [Rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) dataset from Kaggle where we will predict whether it will rain today or not based on some weather measurements. In this section, we will focus on preprocessing by utilizing Scikit-Learn Pipelines.

In [37]:
import pandas as pd

rain = pd.read_csv("data/weatherAUS.csv")
rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [38]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

The dataset contains weather measures of 10 years from multiple weather stations in Australia. You can either predict whether it will rain tomorrow or today, so there are two features in the dataset named `RainToday`, `RainTomorrow`.

Since we will only be predicting for `RainToday`, we will drop the other one along with some other features that won't be necessary:

In [39]:
cols_to_drop = ["Date", "Location", "RainTomorrow", "Rainfall"]

rain.drop(cols_to_drop, axis=1, inplace=True)

> Dropping the `Rainfall` column is a must because it records the amount of rain in millimeters. 

Next, let's deal with missing values starting by looking at their proportions in each column:

In [40]:
missing_props = rain.isna().mean(axis=0)

missing_props

MinTemp          0.010209
MaxTemp          0.008669
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
dtype: float64

If the proportion is higher than 40% we will drop the column:

In [41]:
over_threshold = missing_props[missing_props >= 0.4]

over_threshold

Evaporation    0.431665
Sunshine       0.480098
Cloud3pm       0.408071
dtype: float64

Three columns contain more than 40% missing values:

In [42]:
rain.drop(over_threshold.index, axis=1, inplace=True)

Now, before we move on to pipelines, let's divide the data into feature and target arrays beforehand:

In [43]:
X = rain.drop("RainToday", axis=1)
y = rain.RainToday

Next, there are both categorical and numeric features. We will build two separate pipelines and combine them later. 

> The next code examples will heavily use Sklearn-Pipelines. If you are not familiar with them, check out my separate article for the [complete guide](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d) on them.

For the categorical features, we will impute the missing values with the mode of the column and encode them with One-Hot encoding:

In [44]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

categorical_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("oh-encode", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

For the numeric features, I will choose the mean as an imputer and `StandardScaler` so that the features has 0 mean and a variance of 1:

In [45]:
from sklearn.preprocessing import StandardScaler

numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), ("scale", StandardScaler())]
)

Finally, we will combine the two pipelines with a column transformer. To specify which columns the pipelines are designed for, we should first isolate the categorical and numeric feature names:

In [46]:
cat_cols = X.select_dtypes(exclude="number").columns
num_cols = X.select_dtypes(include="number").columns

Next, we will input these along with their corresponding pipelines into a `ColumnTransFormer` instance:

In [47]:
from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)

The full pipeline is finally ready. The only thing missing is the XGBoost classifier, which we will add in the next section.

### An Example of XGBoost For a Classification Problem

To get started with `xgboost`, just install it either with `pip` or `conda`:

```python
# pip
pip install xgboost

# conda
conda install -c conda-forge xgboost
```

After installation, you can import it under its standard alias - `xgb`. For classification problems, the library provides `XGBClassifier` class:

In [48]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

print(type(xgb_cl))

<class 'xgboost.sklearn.XGBClassifier'>


Fortunately, the classifier follows the familiar fit-predict pattern of `sklearn` meaning we can freely use it as any `sklearn` model. 

Before we train the classifier, let's preprocess the data and divide it into train and test sets:

In [49]:
# Apply preprocessing
X_processed = full_processor.fit_transform(X)
y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
    y.values.reshape(-1, 1)
)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

Since the target contains `NaN`, I imputed it by hand. Also, it is important to pass `y_processed` to `stratify` so that the split contains the same proportion of categories in both sets.

Now, we fit the classifier with default parameters and evaluate its performance:

In [50]:
%%time

from sklearn.metrics import accuracy_score

# Init classifier
xgb_cl = xgb.XGBClassifier()
# Fit
xgb_cl.fit(X_train, y_train)
# Predict
preds = xgb_cl.predict(X_test)
# Score
accuracy_score(y_test, preds)

[11:09:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 14.1 s


0.8507080984463082

Even with default parameters, we got a 85% accuracy which is reasonably good. In the next sections, we will try to improve the model even further by using `GridSearchCV` offered by Scikit-learn.

### What Powers XGBoost Under the Hood

Unlike many other algorithms, XGBoost is am ensemble learning algorithm meaning that it combines the results of many models, called **base learners** to make a prediction. 

Just like in Random Forests, XGBoost uses Decision Trees as base learners:

![tree](./images/tree.png)

An example of a decision tree can be see above. In each decision node (circles), there is a single question that is being asked with only two possible answers. At the bottom of each tree, there is a single decision that is possible (rectangles). In the above tree of whether it rains or not, the first question is whether it is sunny or not. If yes, you immediately decide that it is not going to rain. If otherwise, you continue to ask more binary (yes/no) questions that are ultimately will lead to some decision at the last "leaf" (rectangle).

Individual decision trees are low-bias, high-variance models. They are incredibly good at finding the relationships in any type of training data but struggle to generalize well on unseen data. 

However, the trees used by XGBoost are a bit different than traditional decision trees. They are called CART trees (Classification and Regression trees) and instead of containing a single decision in each "leaf" node, they contain real-value scores of whether an instance belongs to a group. After the tree reaches max depth, the decision can be made by converting the scores into categories using a certain threshold.

I am in no way an expert when it comes to the internals of XGBoost. That's why I recommend you to check out [this awesome YouTube playlist](https://www.youtube.com/playlist?list=PLblh5JKOoLULU0irPgs1SnKO6wqVjKUsQ) entirely on XGBoost and [another one](https://www.youtube.com/playlist?list=PLblh5JKOoLUJjeXUvUE0maghNuY2_5fY6) solely aimed at Gradient Boosting which I did not mention at all.

### Overview of XGBoost Classifier Hyperparameters

So far, we have been using only the default hyperparameters of XGBoost Classifier:

In [51]:
xgb_cl

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

> Terminology refresher: *hyperparameters* of a model are the settings of that model which should be provided by the user. The model itself cannot learn these from the given training data.

It has quite a few as you can see. Even though, we achieved reasonably good results with the defaults, tuning the above parameters might result in significant increase in performance. But before we get to tuning, let's look at the overview of the most frequently tuned hyperparameters:

1. `learning_rate`: also called *eta*, it specifies how quickly the model fits the residual errors by using additional base learners.
- typical values: 0.01 - 0.2
2. `gamma`, `reg_alpha`, `reg_lambda`: these 3 parameters specify the values for 3 types of regularization done by XGBoost - minimum loss reduction to create new split, L1 reg on leaf weights, L2 reg leaf weights respectively
- typical values for `gamma`: 0 - 0.5 but highly dependent on the data
- typical values for `reg_alpha` and `reg_lambda`: 0 - 1 is a good starting point but again, depends on the data
3. `max_depth` - how deep the tree's decision nodes can go. Must be a positive integer
- typical values: 1 - 10
4. `subsample` - fraction of the training set that can be used to train each tree. If this value is low, it may lead to underfitting or if it is too high, it may lead to overfitting
- typical values: 0.5 - 0.9
5. `colsample_bytree` - fraction of the features that can be used to train each tree. Large value means almost all features can be used to build the decision tree
- typical values: 0.5 - 0.9

The above are the main hyperparameters people often tune. It is perfectly OK if you don't understand them all completely (like me) but you can refer to this [post](http://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/) which gives a thorough overview of how each of the above parameters work and how to tune them.

### Hyperparameter Tuning of XGBoost with RandomSearchCV and GridSearchCV

Finally, it is time to super-charge our XGBoost classifier. We will be using the `GridSearchCV` class from Scikit-learn which accepts possible values for desired hyperparameters and fits separate models on the given data for each combination of hyperparameters. I won't go into detail of how GridSearch works but you can check out my separate comprehensive article on the topic:

https://towardsdatascience.com/automatic-hyperparameter-tuning-with-sklearn-gridsearchcv-and-randomizedsearchcv-e94f53a518ee

We won't specify all the hyperparameters at once in the parameter grid. Instead, we will gradually build up by tuning one or two at a time. The reason is that throwing the possible combinations all at once and going through them exhaustively can take days. 

For example, a parameter grid with only 4 parameters and each having 4 possible values will become 4 * 4 * 4 * 4 = 256 combinations and multiplying it by 5-fold cross validation turns it into 1280 rounds. That type of search can take several hours. 

Therefore, we will only be tuning a few of them in two rounds. Let's create the parameter grid for the first one:

In [57]:
param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

In the grid, I fixed `subsample` and `colsample_bytree` to recommended values to speed things up and prevent overfitting.

We will import `GridSearchCV` from `sklearn.model_selection`, instantiate and fit it to our preprocessed data:

In [61]:
from sklearn.model_selection import GridSearchCV

# Init classifier
xgb_cl = xgb.XGBClassifier(objective="binary:logistic")

# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
_ = grid_cv.fit(X_processed, y_processed)

[14:05:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


After excruciatingly long time, we finally got the best params and best score:

In [64]:
grid_cv.best_score_

0.853810061069393

This time, I chose `roc_auc` metric which calculates area under the ROC (receiver operating characteristic) curve. It is one of most popular and robust evaluation metrics for unbalanced classification problems. You can learn more about it [here](https://www.youtube.com/watch?v=4jRBRDbJemM&t=47s). Let's see the best params:

In [65]:
grid_cv.best_params_

{'gamma': 1,
 'learning_rate': 0.1,
 'max_depth': 7,
 'reg_lambda': 10,
 'scale_pos_weight': 3}

As you can see, only `scale_pos_weight` is in the middle of its provided range. The other parameters are at the end of their ranges meaning that we have to keep exploring:

In [76]:
# Insert the new fixed values to the grid
param_grid['scale_pos_weight'] = [3]
param_grid['subsample'] = [0.8]
param_grid['colsample_bytree'] = [0.5]

# Give new value ranges to other params
param_grid['gamma'] = [3, 5, 7]
param_grid['max_depth'] = [9, 15, 20]
param_grid['reg_lambda'] = [10, 30, 50]
param_grid['learning_rate'] = [0.3, 0.5, 0.7, 1]

We will fit a new GridSearch object to the data with the updated param grid:

In [ ]:
%%time

grid_cv_2 = GridSearchCV(xgb_cl, param_grid, cv=3, scoring='roc_auc', n_jobs=-1)

_ = grid_cv_2.fit(X_processed, y_processed)